**Notebook description:**

This notebook was used to create [the updated version][1] of [the original avocado dataset][2] by merging the data from the original dataset and the new data for 2018 -- 2020 downloaded from [the Hass Avocado Board website][3]. For more information on how the updated dataset differs from the original one, please refer to [this][1] page.

[1]: https://www.kaggle.com/timmate/avocado-prices-2020
[2]: https://www.kaggle.com/neuromusic/avocado-prices
[3]: https://hassavocadoboard.com/category-data/

In [ ]:
import pandas as pd

In [ ]:
# Put csv files with the new data in that directory.
DATASETS_DIR = 'new_avocado_data'

## Read in and preprocess Justin's dataset

In [ ]:
original_dataset_path = 'avocado.csv'
original_df = pd.read_csv(original_dataset_path,
                          parse_dates=['Date'],
                          index_col='Date')

original_df.sort_values(['Date', 'region'], inplace=True)
original_df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Lower the columns' names and replace spaces with underscores.
original_df.rename(lambda col_name: col_name.lower().replace(' ', '_'),
                   axis=1, inplace=True)

original_df.rename({'averageprice': 'average_price', 'region': 'geography'},
                   axis=1, inplace=True)

# Remove the existing data from 2018 as there is just little of data for that
# year, hence, we have to add the 2018 data from the Hass Avocado Board CSV
# file. If we do not remove the existing entries for the 2018, we will get
# duplicates after merging the original dataset with the complete 2018 data.
original_df = original_df.query('year != 2018')
original_df.index.name = 'date'

Rename values in the `region` column so that they match the names from the original Hass Avocado Board data, e.g. `SanFrancisco` will be renamed to `San Francisco` and `BaltimoreWashington` will be renamed to `Baltimore/Washington`.

In [ ]:
import os

# Read one of the datasets with the original geographical names of places.
tmp_df = pd.read_csv(os.path.join(DATASETS_DIR, '2018-plu-total-hab-data.csv'))

# The number of geographical names should be the same in both datasets and
# should equal to 54.
assert original_df.geography.nunique() == tmp_df.Geography.nunique(), \
    'number of geographical names is not the same'

# Create a dictionary for renaming the geographical names in the original
# dataset.
renamed_regions_dict = {}
n_regions = tmp_df.Geography.nunique()  # should be 54
correct_region_names = sorted(tmp_df.Geography.unique())
incorrect_region_names = sorted(original_df.geography.unique())

for i in range(n_regions):
    correct_region_name = correct_region_names[i]
    incorrect_region_name = incorrect_region_names[i]

    if correct_region_name == incorrect_region_name:
        # If the correct name is the same as the incorrect one, just skip.
        continue

    else:
        # Add a pair like 'SanFrancisco - San Francisco' to the renaming dict.
        renamed_regions_dict[incorrect_region_name] = correct_region_name

# Rename the geographical names in the original dataset.
original_df.geography.replace(renamed_regions_dict, inplace=True)

## Read in the new data for 2018 -- 2020 and merge it with  Justin's dataset

In [ ]:
# Define a dictionary for renaming names of columns of the new datasets.
RENAMED_COLS_DICT = {
    'asp_current_year': 'average_price',
    'total_bulk_and_bags_units': 'total_volume',
    '4046_units': '4046',
    '4225_units': '4225',
    '4770_units': '4770',
    'totalbagged_units': 'total_bags',
    'smlbagged_units': 'small_bags',
    'lrgbagged_units': 'large_bags',
    'x-lrgbagged_units': 'xlarge_bags'
}

cat_df = original_df.copy()  # a df for concatenating (i.e., merging) the data

print("original dataset's shape:",  original_df.shape)
print()

# Define an accumulatator of the number of entries. This accumulator is used
# for the testing purposes at the bottom of this cell.
n_total_entries_accumulator =  original_df.shape[0]

filenames = sorted(os.listdir(DATASETS_DIR))

for filename in filenames:
    base_filename, extension = filename.split('.')

    if extension != 'csv':
        continue

    else:
        print('processing', filename, '...')

        file_path = os.path.join(DATASETS_DIR, filename)
        tmp_df = pd.read_csv(file_path,
                             parse_dates=['Current Year Week Ending'],
                             index_col='Current Year Week Ending')

        print("dataset's shape:", tmp_df.shape)

        tmp_df.drop('Timeframe', axis=1, inplace=True)
        tmp_df.sort_values(['Current Year Week Ending', 'Geography'], inplace=True)

        # Lower the columns' names and replace spaces with underscores.
        tmp_df.rename(lambda col_name: col_name.lower().replace(' ', '_'), axis=1, inplace=True)
        tmp_df.rename(RENAMED_COLS_DICT, axis=1, inplace=True)
        tmp_df.index.name = 'date'
        tmp_df.type.replace('Conventional ', 'Conventional', inplace=True)
        assert tmp_df.type.nunique() == 2, 'dataset` must contain only 2 types of avocados'

        tmp_df.type = tmp_df.type.apply(lambda avocado_type: avocado_type.lower())
        tmp_df['year'] = tmp_df.index.year

        print('adding', tmp_df.shape[0], 'entries to the original dataset...')
        print()

        cat_df = pd.concat([cat_df, tmp_df], axis=0)
        n_total_entries_accumulator += tmp_df.shape[0]

print("final merged dataset's shape:", cat_df.shape)
print('number of entries in the merged dataset should be:', n_total_entries_accumulator)

assert cat_df.geography.nunique() == 54, 'merged dataset must have 54 unique geographical names'

original dataset's shape: (16953, 12)

processing 2018-plu-total-hab-data.csv ...
dataset's shape: (5292, 12)
adding 5292 entries to the original dataset...

processing 2019-plu-total-hab-data.csv ...
dataset's shape: (5616, 12)
adding 5616 entries to the original dataset...

processing 2020-plu-total-hab-data.csv ...
dataset's shape: (2160, 12)
adding 2160 entries to the original dataset...

final merged dataset's shape: (30021, 12)
number of entries in the merged dataset should be: 30021


## Save the updated (merged) dataset

In [ ]:
# Save with index as it contains the dates.
cat_df.to_csv('avocado-updated-2020.csv')